In [1]:
%pip install -q selenium webdriver_manager

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\administrator\appdata\local\programs\python\python312\lib\site-packages\torchlight-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


## 한번만 실행

In [2]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 구글 로그인 페이지로 이동
driver.get('https://accounts.google.com/signin')

# 수동으로 로그인 진행
print("Please log in manually and then press Enter here...")
input("Press Enter after you have logged in...")  # 터미널에서 엔터를 누름

# 로그인된 쿠키 저장
cookies = driver.get_cookies()
with open('cookies.pkl', 'wb') as file:
    pickle.dump(cookies, file)

print("Cookies saved.")
driver.quit()


Please log in manually and then press Enter here...
Cookies saved.


## 크롤링 메인 코드

In [4]:
import subprocess
import time
import psutil
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import json
import pandas as pd
import os
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
chrome_process = subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def wait_for_page_load(driver, timeout=120):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
        return True
    except TimeoutException:
        print("페이지 로딩 시간 초과")
        return False

def scroll_to_element(driver, element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(1)  # 스크롤 후 잠시 대기

def click_element_by_xpath(driver, xpath, max_retries=5):
    for attempt in range(max_retries):
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )
            scroll_to_element(driver, element)
            element.click()
            return True
        except Exception as e:
            print(f"Element click error (Attempt {attempt+1}): {e}")
            time.sleep(2)
    return False

def kill_chrome_debugging_process(port):
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        if 'chrome' in proc.info['name'] and f'--remote-debugging-port={port}' in ' '.join(proc.info['cmdline']):
            proc.terminate()
            proc.wait()
            print(f"Chrome debugging process on port {port} terminated.")

def process_title(driver, index, titles, max_retries=3):
    simple_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{index+1}]/div/div[2]/button'
    detail_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{index+1}]/div[2]/div[2]/button'
    
    for attempt in range(max_retries):
        try:
            # 간단 텍스트 버튼 클릭
            if click_element_by_xpath(driver, simple_xpath):
                print(f"간단 텍스트 버튼 클릭됨: {index}")
                time.sleep(3)

                # 간단 텍스트 추출
                simple_text_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{index+1}]/div[2]/div[1]'
                simple_text_elements = driver.find_elements(By.XPATH, simple_text_xpath)
                simple_text = "\n".join([element.text for element in simple_text_elements])

                # 자세히 버튼 클릭
                if click_element_by_xpath(driver, detail_xpath):
                    print(f"자세히 버튼 클릭됨: {index}")
                    time.sleep(3)

                    # 자세히 텍스트 추출
                    detail_text_xpath = f'//*[@id="app"]/div/div[1]/div[2]/div[2]/div/div[{index+1}]/div[2]/div[2]/div'
                    detail_text_elements = driver.find_elements(By.XPATH, detail_text_xpath)
                    detail_text = "\n".join([element.text for element in detail_text_elements])

                    return {
                        'SUMMARY_TITLE': titles[index].text if index < len(titles) else f"Summary {index+1}",
                        'SIMPLE_TEXT': simple_text,
                        'DETAIL_TEXT': detail_text
                    }
                else:
                    print(f"자세히 버튼 클릭 실패: {index}")
            else:
                print(f"간단 텍스트 버튼 클릭 실패: {index}")
            
            # 클릭 실패 시 재시도를 위해 예외 발생
            raise Exception("Button click failed")
        
        except Exception as e:
            print(f"Error processing title {index} (Attempt {attempt+1}): {e}")
            if attempt == max_retries - 1:
                print(f"Failed to process title {index} after {max_retries} attempts")
                return None
            time.sleep(2)  # 재시도 전 대기

try:
    # 데이터 저장 리스트 및 실패한 URL 저장 리스트
    all_data = {}
    failed_urls = []

    # URL 리스트 경로
    path_list_file = r'D:\hackerton_6th\team-project\src\crawling\1_Data_Youtube URL\Dog\강형욱유튜브url모음\기초교육.csv'
    df = pd.read_csv(path_list_file)
   
    # url 몇개까지 할건지 개수지정. 앨런=하루 30개 제한, ex) [:30], [31:60], [61:90]...
    path_list = df['링크'].tolist()[20:22]  #첫째날, [0:30] / 둘째날 [31:60] 이런식으로 30개씩 진행.

    # 각 URL에 대해 반복 작업 수행
    for url in path_list:
        print(f"처리 중인 URL: {url}")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                driver.get('https://alan.est.ai/youtube?showHistoryMenu=false&showLeftSidebar=true')
                if not wait_for_page_load(driver):
                    raise TimeoutException("페이지 로딩 실패")

                textarea = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'textarea[placeholder*="요약할 유튜브 링크를 입력해주세요"]'))
                )
                textarea.clear()
                textarea.send_keys(url)
                textarea.send_keys(Keys.RETURN)

                # 요약 완료 메시지 대기
                try:
                    WebDriverWait(driver, 180).until(
                        EC.presence_of_element_located((By.XPATH, '//p[contains(text(), "영상 요약을 완료했습니다!")]'))
                    )
                    print(f"요약 완료 메시지가 표시되었습니다: {url}")
                except TimeoutException:
                    print(f"3분 이상 무반응: {url} 처리 실패")
                    failed_urls.append(url)  # 실패한 URL 추가
                    break  # 다음 URL로 이동

                time.sleep(10)

                # 타이틀 추출
                titles = driver.find_elements(By.CSS_SELECTOR, 'div._title_1skjv_47')
                print(f"타이틀 개수: {len(titles)}")
                time.sleep(5)

                url_data = {
                    'URL': url,
                    'SUMMARIES': []
                }

                processed_indices = set()
                max_overall_retries = 3

                for overall_attempt in range(max_overall_retries):
                    for i in range(len(titles)):
                        if i not in processed_indices:
                            result = process_title(driver, i, titles)
                            if result:
                                url_data['SUMMARIES'].append(result)
                                processed_indices.add(i)
                    
                    if len(processed_indices) == len(titles):
                        print("모든 타이틀이 성공적으로 처리되었습니다.")
                        break
                    else:
                        print(f"처리되지 않은 타이틀이 있습니다. 재시도 중... (시도 {overall_attempt + 1}/{max_overall_retries})")
                        time.sleep(5)  # 전체 재시도 전 대기

                if len(processed_indices) < len(titles):
                    print(f"경고: {len(titles) - len(processed_indices)}개의 타이틀을 처리하지 못했습니다.")

                all_data[url] = url_data
                break  # URL 처리 성공, 다음 URL로 이동

            except Exception as e:
                print(f"Error processing URL {url} (Attempt {attempt+1}): {e}")
                if attempt == max_retries - 1:
                    print(f"Failed to process URL {url} after {max_retries} attempts")
                    failed_urls.append(url)
                time.sleep(5)  # URL 처리 실패 시 대기

    # JSON 파일로 저장
    try:
        with open('test_3.json', 'w', encoding='utf-8') as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)
        print("데이터가 extracted_data.json 파일에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}")

    # 실패한 URL 저장
    if failed_urls:
        with open('failed_urls.txt', 'w') as f:
            for url in failed_urls:
                f.write(url + '\n')
        print("실패한 URL들이 failed_urls.txt 파일에 저장되었습니다.")

finally:
    # 드라이버 종료
    driver.quit()

    # Chrome 디버깅 프로세스 종료
    kill_chrome_debugging_process(chrome_debug_port)

처리 중인 URL: https://www.youtube.com/watch?v=bMfCNGz0WDM&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=21&pp=iAQB
요약 완료 메시지가 표시되었습니다: https://www.youtube.com/watch?v=bMfCNGz0WDM&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=21&pp=iAQB
타이틀 개수: 1
간단 텍스트 버튼 클릭됨: 0
자세히 버튼 클릭됨: 0
모든 타이틀이 성공적으로 처리되었습니다.
처리 중인 URL: https://www.youtube.com/watch?v=bgUup3C-LdA&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=22&pp=iAQB
요약 완료 메시지가 표시되었습니다: https://www.youtube.com/watch?v=bgUup3C-LdA&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=22&pp=iAQB
타이틀 개수: 1
간단 텍스트 버튼 클릭됨: 0
자세히 버튼 클릭됨: 0
모든 타이틀이 성공적으로 처리되었습니다.
데이터가 extracted_data.json 파일에 저장되었습니다.
Chrome debugging process on port 9222 terminated.


## Json 파일 병합

In [5]:
import json
import os

# JSON 파일 모아둔 폴더 경로
folder_path = '기초교육'  # 예: './json_files'

merged_data = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            merged_data.update(data)

# 병합된 데이터를 새로운 JSON 파일로 저장
output_file_path = os.path.join(folder_path, 'merged_file.json')
with open(output_file_path, 'w', encoding='utf-8') as outfile:
    json.dump(merged_data, outfile, ensure_ascii=False, indent=4)

print("JSON 파일이 성공적으로 병합되었습니다.")


JSON 파일이 성공적으로 병합되었습니다.
